### Decision Tree Basics

In [12]:
import numpy as np

In [13]:
data = np.array([
    ['Pointy', 'Round',     'Present', 1],
    ['Floppy', 'Not round', 'Present', 1],
    ['Floppy', 'Round',     'Absent',  0],
    ['Pointy', 'Not round', 'Present', 0],
    ['Pointy', 'Round',     'Present', 1],
    ['Pointy', 'Round',     'Absent',  1],
    ['Floppy', 'Not round', 'Absent',  0],
    ['Pointy', 'Round',     'Absent',  1],
    ['Floppy', 'Round',     'Absent',  0],
    ['Floppy', 'Round',     'Absent',  0],
])

X, Y = data[:, :-1], data[:, -1]
Y = Y.reshape((-1, 1))

In [16]:
class Node:
    def __init__(self, data = None, children = None, split_on = None, pred_class = None, is_leaf = False):
        """
        Parameters
        ----------
        data: numpy.ndarray, default=None
            Stores corresponding data in each node.
        children: dict(feat_value: Node), default=None
            Contains all direct nodes after data is split into nodes.
        split_on: int, default=None
            Shows which feature, the model decides to split.
        pred_class : str, default=None
            The predicted class for the node (only applicable to leaf nodes)
        is_leaf: bool, default=False
            Demonstrates whether the node is a leaf or not!
        """

        self.data = data
        self.children = children
        self.split_on = split_on
        self.pred_class = pred_class
        self.is_leaf = is_leaf

    
class DecisionTreeClassifier:
    def __init__(self):
        self.root = Node()

    
    @staticmethod
    def calculate_entropy(Y):
        """
        Parameters:
        -----------
        Y: numpy.ndarray
            The labels array.

        Returns:
        -----------
        entropy: flaot
            The entropy value of the given labels.
        """
        _, labels_count = np.unique(Y, return_counts=True)
        total_instances = len(Y)

        entropy = sum([-label_count/total_instances * np.log2(label_count/total_instances) for label_count in labels_count])
        return entropy
    
    def split_on_feature(self, data, feat_index):
        """
        Split the dataset based on a specific feature index.

        Parameters:
        -----------
        data: numpy.ndarray
            The dataset to be split.
        feat_index: int
            The index of the feature to perform the split.
        Returns:
        -----------
        - split_nodes: dict
            A dictionary of split nodes. 
            (feature value as key, corresponding node as value)
        - weighted_entropy: float
            The weighted entropy of the split.
        """
        feature_values = data[:, feat_index]
        unique_values = np.unique(feature_values)

        split_nodes = {}
        weighted_entropy = 0
        total_instances = len(data)

        for unique_value in unique_values:
            partition = data[data[:, feat_index] == unique_value, :]
            node = Node(data = partition)
            split_nodes[unique_value] = node
            partition_y = self.get_y(partition)
            node_entropy = self.calculate_entropy(partition_y)
            weighted_entropy += (len(partition)/total_instances) * node_entropy

        return split_nodes, weighted_entropy
    
    def best_split(self, node):
        """
        Find the best split for the given node.
        (data in node.data)

        Parameters:
        ----------
        node: Node
            The node for which the best split is being determined.

        If the node meets the criteria to stop splitting:
            - Mark the node as a leaf.
            - Assign a predicted class for future predictions based on the target values (y).
            - return.

        Otherwise:
            - Initialize variables for tracking the best split.
            - Iterate over the features to find the best split.
            - Split the data based on each feature and calculate the weighted entropy of the split.
            - Compare the current weighted entropy with the previous best entropy.
            - Update the best split variables if the current split has lower entropy.
            - update the node with the best split information, including child nodes and the feature index used for the split.
            - Recursively call the best_split function for each child node.
        """
        # Check if the node meets stopping criteria
        if self.meet_criteria(node):
            node.is_leaf = True
            y = self.get_y(node.data)
            node.pred_class = self.get_pred_class(y)

            return 
        # Initialize variables to track the best split
        index_feature_split = -1
        min_entropy = 1

        # Iterate over all features (ignore y)
        for i in range(data.shape[1] - 1):
            split_nodes, weighted_entropy = self.split_on_feature(node.data, i)
            if weighted_entropy < min_entropy:
                child_nodes, min_entropy = split_nodes, weighted_entropy
                index_feature_split = i

        node.children = child_nodes
        node.split_on = index_feature_split

        # Recursively call the best_split function for each child node
        for child_node in child_nodes.values():
            self.best_split(child_node)
        
    def meet_criteria(self, node):
        """
        Check if the criteria for stopping the tree expansion is met for a given node. Here we only check if the entropy of the target values (y) is zero.
        Additionally, you can customize criteria based on your specific requirements. For instance, you can set the maximum depth for the decision tree or incorporate other conditions for stopping the tree expansion. Modify the implementation of this method according to your desired criteria.

        Parameters:
        -----------
        node : Node
            The node to check for meeting the stopping criteria.

        Returns:
        -----------
        bool
            True if the criteria is met, False otherwise.
        """

        y = self.get_y(node.data)
        return True if self.calculate_entropy(y) == 0 else False
    
    @staticmethod
    def get_y(data):
        """
        Get the target (y) from the data.
        """
        y = data[:, -1]
        return y
    
    @staticmethod
    def get_pred_class(Y):
        """
        Get the predicted class label based on the majority vote.
        """
        labels, labels_counts = np.unique(Y, return_counts=True)
        index = np.argmax(labels_counts)

        return labels[index]
    
    def fit(self, X, Y):
        data = np.column_stack([X, Y])
        self.root.data = data
        self.best_split(self.root)

    def predict(self, X):
        predictions = np.array([self.traverse_tree(x, self.root) for x in X])
        return predictions
    
    def traverse_tree(self, x, node):
        """
        Recursively traverse the decision tree to predict the class label for a given input.

        Parameters:
        -----------
        x:
            The input for which to make a prediction.

        node:
            The current node being traversed in the decision tree.
        """

        # check if the current node is a leaf node
        if node.is_leaf:
            return node.pred_class
        
        # get the feature value at the split point for the current node
        feat_value = x[node.split_on]

        # Recursively traverse the decision tree using the child node corresponding to the feature value
        predicted_class = self.traverse_tree(x, node.children[feat_value])
        return predicted_class
    

    # share github
    # interpretability and explainability
    # non-parametric
    # inference time
    # general overview of the decision trees
    # gradient boosting implement
    #  

In [17]:
model = DecisionTreeClassifier()
model.fit(X, Y)

model.predict(
    [['Pointy', 'Round',     'Present'],
    ['Floppy', 'Not round', 'Present'],
    ['Floppy', 'Round',     'Absent']
]) # ['1' '1' '0']

array(['1', '1', '0'], dtype='<U1')

In [18]:
index_to_name = {0: 'Ear Shape', 1: 'Face Shape', 2: 'Whiskers'}

root = model.root
print("Root")
# Root
print(f"- split on: {index_to_name[root.split_on]}" )
# - split on: Ear Shape
print(f"- children: {root.children}", '\n')          
# - children: {'Floppy': <__main__.Node object at 0x7f87c94d0760>, 'Pointy': <__main__.Node object at 0x7f87c94d01c0>} 

pointy_node = root.children['Pointy']
print("Pointy Node")
# Pointy Node
print(f"- split on: {index_to_name[pointy_node.split_on]}" )
# - split on: Face Shape
print(f"- children: {pointy_node.children}")
# - children: {'Not round': <__main__.Node object at 0x7f87c94d07f0>, 'Round': <__main__.Node object at 0x7f87c94d0e80>}
print(f"- data: \n {pointy_node.data}", '\n')
# All pointy data

floppy_node = root.children['Floppy']
print("Floppy Node")
# Floppy Node
print(f"- split on: {index_to_name[floppy_node.split_on]}" )
# - split on: Whiskers
print(f"- children: {floppy_node.children}")
# - children: {'Absent': <__main__.Node object at 0x7f87c94d09d0>, 'Present': <__main__.Node object at 0x7f87c94d0400>}
print(f"- data: \n {floppy_node.data}", '\n')
# All floppy data

Root
- split on: Ear Shape
- children: {np.str_('Floppy'): <__main__.Node object at 0x1668a3ee0>, np.str_('Pointy'): <__main__.Node object at 0x166892bb0>} 

Pointy Node
- split on: Face Shape
- children: {np.str_('Not round'): <__main__.Node object at 0x165f97d90>, np.str_('Round'): <__main__.Node object at 0x165f97d00>}
- data: 
 [['Pointy' 'Round' 'Present' '1']
 ['Pointy' 'Not round' 'Present' '0']
 ['Pointy' 'Round' 'Present' '1']
 ['Pointy' 'Round' 'Absent' '1']
 ['Pointy' 'Round' 'Absent' '1']] 

Floppy Node
- split on: Whiskers
- children: {np.str_('Absent'): <__main__.Node object at 0x1669a2a00>, np.str_('Present'): <__main__.Node object at 0x165f97d30>}
- data: 
 [['Floppy' 'Not round' 'Present' '1']
 ['Floppy' 'Round' 'Absent' '0']
 ['Floppy' 'Not round' 'Absent' '0']
 ['Floppy' 'Round' 'Absent' '0']
 ['Floppy' 'Round' 'Absent' '0']] 

